In [7]:
import pandas as pd
import pyarrow.parquet as pa

In [24]:
data=pd.read_parquet('train-00000-of-00001-090b52ccb189d47a.parquet',engine='pyarrow')

In [25]:
data.head()

,text
0,April
1,"April is the fourth month of the year, and com..."
2,April always begins on the same day of week as...
3,April's flowers are the Sweet Pea and Daisy. I...
4,"April comes between March and May, making it t..."


In [28]:
data.head(100)

,text
0,April
1,"April is the fourth month of the year, and com..."
2,April always begins on the same day of week as...
3,April's flowers are the Sweet Pea and Daisy. I...
4,"April comes between March and May, making it t..."
5,April begins on the same day of the week as Ju...
6,"In common years, April starts on the same day ..."
7,"In years immediately before common years, Apri..."
8,April is a spring month in the Northern Hemisp...
9,It is unclear as to where April got its name. ...
